### Let's try cox survival analysis here with a reduced dataset

In [ ]:
import pickle
import pandas as pd

In [62]:
#load initial cleaned data
df=pd.read_csv(r"..\data\processed\prelim_clean2.csv", index_col=[0])
df.drop(['visit_interval'],axis=1, inplace=True)
df2=pd.read_csv(r"..\data\processed\prelim_clean2.csv", index_col=[0])

In [63]:
broad_features = pickle.load(open('../models/01final_features_broad.sav', 'rb'))
res_features = pickle.load(open('../models/01final_features_res.sav', 'rb'))

In [64]:
broad_subset=df[df.columns.intersection(broad_features)]
subset=df[df.columns.intersection(res_features)]

In [73]:
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation

broad_subset['visit_interval']=df2['visit_interval']
broad_subset['target']=df2['target']

cph = CoxPHFitter(penalizer=0.1)
#scores = k_fold_cross_validation(cph, broad_subset, 'visit_interval', event_col='target', k=10)
#print(scores)

# scores = k_fold_cross_validation(cph, broad_subset, duration_col='visit_interval', event_col='target', k=10, scoring_method="concordance_index")
# print(scores)

cph.fit(df=broad_subset, duration_col='visit_interval', event_col='target')
cph.print_summary()


<lifelines.CoxPHFitter: fitted with 5220 total observations, 2977 right-censored observations>
             duration col = 'visit_interval'
                event col = 'target'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 5220
number of events observed = 2243
   partial log-likelihood = -16627.84
         time fit was run = 2020-09-29 03:16:15 UTC

---
             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                          
BIRTHYR     -0.01       0.99       0.00            -0.01             0.00                 0.99                 1.00
INDEPEND     0.06       1.06       0.03             0.00             0.12                 1.00                 1.13
NACCAM       0.03       1.03       0.00             0.02             0.03                 1.02                 1.03
WEIGHT      -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
MEMORY       0.08       1.08       0.14            -0.19             0.34                 0.83                 1.41
MEMORY       0.08       1.08       0.14            -0.19             0.34                 0.83                 1.41
MEMORY       0.08       1.08       0.14            -0.19             0.34                 0.83                 1.41
MEMORY       0.08       1.08       0.14            -0.19             0.34                 0.83                 1.41
ORIENT       0.06       1.06       0.12            -0.17             0.29                 0.84                 1.33
ORIENT       0.06       1.06       0.12            -0.17             0.29                 0.84                 1.33
ORIENT       0.06       1.06       0.12            -0.17             0.29                 0.84                 1.33
ORIENT       0.06       1.06       0.12            -0.17             0.29                 0.84                 1.33
JUDGMENT     0.06       1.06       0.08            -0.10             0.22                 0.90                 1.25
COMMUN       0.03       1.03       0.13            -0.22             0.29                 0.80                 1.33
COMMUN       0.03       1.03       0.13            -0.22             0.29                 0.80                 1.33
COMMUN       0.03       1.03       0.13            -0.22             0.29                 0.80                 1.33
COMMUN       0.03       1.03       0.13            -0.22             0.29                 0.80                 1.33
HOMEHOBB     0.01       1.01       0.12            -0.22             0.24                 0.80                 1.28
HOMEHOBB     0.01       1.01       0.12            -0.22             0.24                 0.80                 1.28
HOMEHOBB     0.01       1.01       0.12            -0.22             0.24                 0.80                 1.28
HOMEHOBB     0.01       1.01       0.12            -0.22             0.24                 0.80                 1.28
PERSCARE    -0.11       0.90       0.09            -0.29             0.07                 0.75                 1.07
CDRSUM       0.01       1.01       0.03            -0.05             0.07                 0.95                 1.08
CDRSUM       0.01       1.01       0.03            -0.05             0.07                 0.95                 1.08
CDRSUM       0.01       1.01       0.03            -0.05             0.07                 0.95                 1.08
CDRSUM       0.01       1.01       0.03            -0.05             0.07                 0.95                 1.08
CDRGLOB     -0.10       0.90       0.15            -0.39             0.19                 0.68                 1.20
COMPORT     -0.03       0.97       0.02            -0.08             0.02                 0.92                 1.02
CDRLANG     -0.01       0.99       0.02            -0.06             0.03         

In [82]:
subset['visit_interval']=df2['visit_interval']
subset['target']=df2['target']

cph = CoxPHFitter(penalizer=0.01)
scores = k_fold_cross_validation(cph, subset, duration_col='visit_interval', event_col='target', k=10, scoring_method="concordance_index")
print(scores)

cph.fit(df=broad_subset, duration_col='visit_interval', event_col='target')
#cph.print_summary()

[0.660089377793056, 0.6547516607579251, 0.6457405089695453, 0.6245722290192741, 0.6341896741640906, 0.6575643179059054, 0.6836509595130285, 0.6852457314858063, 0.6576367019753794, 0.6838441473752853]


<lifelines.CoxPHFitter: fitted with 5220 total observations, 2977 right-censored observations>

In [ ]:
# from lifelines.utils.sklearn_adapter import sklearn_adapter

# from lifelines import CoxPHFitter
# from lifelines.datasets import load_rossi

# X = load_rossi().drop('week', axis=1) # keep as a dataframe
# Y = load_rossi().pop('week')

# CoxRegression = sklearn_adapter(CoxPHFitter, event_col='arrest')
# # CoxRegression is a class like the `LinearRegression` class or `SVC` class in scikit-learn

# sk_cph = CoxRegression(penalizer=1e-5)
# sk_cph.fit(X, Y)
# print(sk_cph)

# """
# SkLearnCoxPHFitter(alpha=0.05, penalizer=1e-5, strata=None, tie_method='Efron')
# """

# sk_cph.predict(X)
# sk_cph.score(X, Y)